# **COVID-19 Deaths** #
### September 2020 - Present ###

These graphs are a representation of COVID-19 data in the UK by region, from September 2020 onwards.

If this is your first time viewing these graph the data displayed in them will be a stored representation, and may be out of date. Press refresh and re-select areas for the most recent statistics.  

#### Data Sources ####

All data used is supplied by the UK government via Public Health England.

In [1]:
from uk_covid19 import Cov19API
import json
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as wdg

%matplotlib inline
plt.rcParams['figure.dpi'] = 200

## **Comparing Daily New Deaths And Cases** ##

This graph is a representation of all daily new cases and deaths of COVID-19.

In [2]:
from cases_deaths import start

Button(description='Refresh data', icon='refresh', style=ButtonStyle(), tooltip='Click to download current Pub…

In [3]:
# determines whether json data will be loaded locally (to provide a visual output pre-API call) 
refreshCount = 0

In [4]:
# loads local json file

with open("initialJSON.json", "rt") as INFILE:
    savedJSONdata=json.load(INFILE)

In [5]:
# adds one to refresh count
def refreshCounter():
    global refreshCount
    refreshCount += 1
    return(refreshCount)

In [6]:
# makes API call for cumulative deaths by UK region
def refreshAPI():
    filters = [
    'areaType=region'
]

    structure = {
    'Date': 'date',
    'Area': 'areaName',
    'Deaths': 'cumDeaths28DaysByPublishDate'
}

    global api 
    
    api = Cov19API(filters=filters, structure=structure)

    global areaCompare

    areaCompare = api.get_json()
    return(areaCompare)

In [7]:
# formats API data into json
def newJSON():
    
    areaCompare = refreshAPI()
    
    with open("areaCompare.json", "wt") as OUTF:
        json.dump(areaCompare, OUTF)

    jsonfile = open("areaCompare.json", "r")
    data = jsonfile.readline()
    
    with open("areaCompare.json", "rt") as INFILE:
        data=json.load(INFILE)
    
    
    return(data)

In [8]:
# determines what json data to display
def dataSelect():
    x = refreshCounter()
    global jsonDF
    if x == 1:
        jsonDF = savedJSONdata
    else:
        jsonDF = newJSON()

In [9]:
dataSelect()

In [10]:
# produces list of dates between 09/2020 and now
def wrangleData(jsonDF):
    global newdates
    global refresh_count
    global datalist
    
    datalist=jsonDF['data']
    dates=[dictionary['Date'] for dictionary in datalist]
    
    dates.sort()
    dates = list(dict.fromkeys(dates))
    
    newdates = []
    
    for date in dates:
        if date[0:7] != '2020-08':
            x = date
            newdates.append(x)
    
    return(newdates)
    #return(datalist)


In [11]:
# puts data into nested dictionary format - {date:[area:deaths, area:deaths..], date:[area:deaths...] etc., 
def datedictionary():
    global newdates
    global datalist
    newdates = wrangleData(jsonDF)
    datedict = {}

    for date in newdates:
        datedict[date] = {}

    for dictionary in datalist:
        for entry in datedict:
            if dictionary['Date'] == entry:
                tempdict = {dictionary['Area']: dictionary['Deaths']}
                datedict[entry].update(tempdict)
    
    return(datedict)

In [12]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [13]:
# sets up the empty dataframe with dates as index
def makeDataframe():
    global datedict
    global areaComparedf
    datedict = datedictionary()
    startdate=parse_date(list(datedict)[0])
    enddate=parse_date(list(datedict)[-1])

    index=pd.date_range(startdate, enddate, freq='D')
    areaComparedf = pd.DataFrame(index=index, columns=['South East', 'Yorkshire and The Humber', 'North East',
                                                 'East Midlands', 'London', 'South West',
                                                 'North West', 'West Midlands',
                                                 'East of England'])
    return areaComparedf

In [14]:
# populates dataframe from nested dictionary
def populateDataframe():
    areaComparedf = makeDataframe()
    for entry in datedict:
        date=parse_date(entry)
    #for dicto in datedict:
        for column in ['South East', 'Yorkshire and The Humber', 'North East',
                                                 'East Midlands', 'London', 'South West',
                                                 'North West', 'West Midlands',
                                                 'East of England']:

            if pd.isna(areaComparedf.loc[date, column]): 

                try:
                    value= (datedict[entry][column]) if (datedict[entry][column])!=None else 0.0
                    areaComparedf.loc[date, column]=value
                except Exception:
                    pass
    
            # fill in any remaining "holes" due to missing dates
    areaComparedf.fillna(0.0, inplace=True)
            
    return(areaComparedf)

In [15]:
# plots graph
def plotData():
    global graph
    data = populateDataframe()
    areaCompare_graph = data.plot()
    graph = (areaCompare_graph)
    return(graph)

In [16]:
# sets up selection widget implements widget on final graph

areaComparedf = populateDataframe()

series=wdg.SelectMultiple(
options=['South East', 'Yorkshire and The Humber', 'North East',
                                                 'East Midlands', 'London', 'South West',
                                                 'North West', 'West Midlands',
                                                 'East of England'],
value=['South East', 'Yorkshire and The Humber', 'North East',
                                                 'East Midlands', 'London', 'South West',
                                                 'North West', 'West Midlands',
                                                 'East of England'],
rows=9,
description='Stats:',
disabled=False
)

controls=wdg.VBox([series])

def setControls(gcols):
    ncols=len(gcols)
    if ncols>0:
        areaComparedf[list(gcols)].plot()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
  

def finalGraph():
    graph=wdg.interactive_output(setControls, {'gcols': series})

    #display(controls, graph)
    combined = wdg.HBox([graph, controls])
    display(combined)

In [17]:
#sets functionality of refresh button, calling API, looping through all steps and adding to refresh counter

def access_api(button):
    refreshAPI()
    dataSelect()
    newJSON()
    wrangleData(jsonDF)
    datedictionary()
    makeDataframe()
    populateDataframe()
    refreshCounter()
    print("I'm downloading data from the API...")
    print("...all done.")

In [18]:
# determines style of refresh button
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='refresh' # (FontAwesome names without the `fa-` prefix)
)

apibutton.on_click(access_api)

In [19]:
def start():
    finalGraph()
    display(apibutton)

## **Comparing Deaths By Region** ##

This graph is a representation of all deaths from COVID-19 across every UK region, measured cumulatively.

In [20]:
start()

Button(description='Refresh data', icon='refresh', style=ButtonStyle(), tooltip='Click to download current Pub…